In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [2]:
from seq import Widget

w = Widget()
w

Widget()

In [1]:
from datasets import load_dataset

ds = load_dataset("LawInformedAI/overruling")

ds

Generating train split:   0%|          | 0/2400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence1'],
        num_rows: 2400
    })
})